In [1]:
# DATA PREPROCESSING

#Import necessary packages.
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#Import necessary data
data2018 = pd.read_csv(r'../Data/FIFAWorldCup2018.csv')
# data2014 = pd.read_csv(r'../Data/FIFAWorldCup2014.csv')
# data2010 = pd.read_csv(r'../Data/FIFAWorldCup2010.csv')
#data2006 = pd.read_csv(r'../Data/FIFAWorldCup2006.csv')

#Stats about data sets
print("Dimensions of FIFA WC 2018: {}rows X {}columns".format(data2018.shape[0],data2018.shape[1]))
# print("Dimensions of FIFA WC 2014: {}rows X {}columns".format(data2014.shape[0],data2014.shape[1]))
# print("Dimensions of FIFA WC 2010: {}rows X {}columns".format(data2010.shape[0],data2010.shape[1]))
# print("Dimensions of FIFA WC 2006: {}rows X {}columns".format(data2006.shape[0],data2006.shape[1]))


Dimensions of FIFA WC 2018: 64rows X 64columns


In [2]:
# DATA INTEGRATION
# dataToConcat = [data2006,data2010,data2014,data2018]
# trainingData = pd.concat([])


In [3]:
# DATA REDUCTION
namesOfColumnsToBeDropped = ["date_GMT","status","attendance","referee","stadium_name", "team_a_xg", "team_b_xg", "btts_percentage_pre_match", 
"over_15_percentage_pre_match", "over_25_percentage_pre_match","over_35_percentage_pre_match", "over_45_percentage_pre_match",
"over_15_HT_FHG_percentage_pre_match", "over_05_HT_FHG_percentage_pre_match", "over_15_2HG_percentage_pre_match", "over_05_2HG_percentage_pre_match","timestamp","Game Week","home_team_goal_timings","away_team_goal_timings",'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg','odds_ft_home_team_win',
 'odds_ft_draw' ,'odds_ft_away_team_win' ,'odds_ft_over15', 'odds_ft_over25',
 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes', 'odds_btts_no','total_goal_count', 'total_goals_at_half_time', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time','home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'away_team_second_half_cards','average_goals_per_match_pre_match', 'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match']

dataFrame = data2018.drop(namesOfColumnsToBeDropped,axis=1)

print("Dimensions of FIFA WC 2018 after dropping attributes: {}rows X {}columns".format(dataFrame.shape[0],dataFrame.shape[1]))

# Uncomment next line to display data after dropping unnecessary columns
display(dataFrame)


Dimensions of FIFA WC 2018 after dropping attributes: 64rows X 20columns


,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession
0,Russia,Saudi Arabia,5,0,6,2,1,0,1,0,11,4,8,0,3,4,22,10,38,62
1,Egypt,Uruguay,0,1,0,5,2,0,0,0,7,12,4,6,3,6,12,6,41,59
2,Morocco,Iran,0,1,5,2,1,0,3,0,10,9,4,3,6,6,22,14,67,33
3,Portugal,Spain,3,3,4,5,1,0,1,0,7,12,4,7,3,5,12,10,33,67
4,France,Australia,2,1,5,1,1,0,3,0,11,5,6,2,5,3,16,19,55,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Russia,Croatia,2,2,6,8,1,0,4,0,13,15,6,4,7,11,25,18,35,65
60,France,Belgium,1,0,4,5,2,0,3,0,14,9,6,4,8,5,6,16,36,64
61,Croatia,England,2,1,8,4,2,0,1,0,19,9,8,3,11,6,23,14,55,45
62,Belgium,England,2,0,4,5,1,0,2,0,8,14,5,7,3,7,11,4,42,58


In [4]:
# Feature Engineering

# print mapping
print(dataFrame["home_team_name"].tolist())

# One-Hot Encoding
labelencoder = LabelEncoder()
dataFrame["home_team_name"] = labelencoder.fit_transform(dataFrame["home_team_name"])
dataFrame["away_team_name"] = labelencoder.fit_transform(dataFrame["away_team_name"])


# Uncomment next line to display data after One-Hot Encoding
#display(dataFrame)

# Adding fields "home_team_result" and "winner_encoded"
winnerEncoded = []
homeTeamResult = []
for index,match in dataFrame.iterrows():
    # when home team wins
    if(match["home_team_goal_count"]>match["away_team_goal_count"]):
        winnerEncoded.append(match["home_team_name"])
        homeTeamResult.append(1.0)

    # when away team wins
    if(match["home_team_goal_count"]<match["away_team_goal_count"]):
        winnerEncoded.append(match["away_team_name"])
        homeTeamResult.append(0.0)

    # draws and tie breaker
    if(match["home_team_goal_count"]==match["away_team_goal_count"]):
        if((match["home_team_goal_count"]/match["home_team_shots"])>=(match["away_team_goal_count"]/match["away_team_shots"])):
            winnerEncoded.append(match["home_team_name"])
            homeTeamResult.append(1.0)
              
        if((match["home_team_goal_count"]/match["home_team_shots"])<(match["away_team_goal_count"]/match["away_team_shots"])):
            winnerEncoded.append(match["away_team_name"])
            homeTeamResult.append(0.0)

dataFrame["winner_encoded"] = winnerEncoded
dataFrame["home_team_result"] = homeTeamResult

# Uncomment the next line to display dataframe after adding fields "home_team_result" and "winner_encoded"
display(dataFrame)


['Russia', 'Egypt', 'Morocco', 'Portugal', 'France', 'Argentina', 'Peru', 'Croatia', 'Costa Rica', 'Germany', 'Brazil', 'Sweden', 'Belgium', 'Tunisia', 'Colombia', 'Poland', 'Russia', 'Portugal', 'Uruguay', 'Iran', 'Denmark', 'France', 'Argentina', 'Brazil', 'Nigeria', 'Serbia', 'Belgium', 'South Korea', 'Germany', 'England', 'Japan', 'Poland', 'Uruguay', 'Saudi Arabia', 'Iran', 'Spain', 'Denmark', 'Australia', 'Nigeria', 'Iceland', 'South Korea', 'Mexico', 'Serbia', 'Switzerland', 'Japan', 'Senegal', 'England', 'Panama', 'France', 'Uruguay', 'Spain', 'Croatia', 'Brazil', 'Belgium', 'Sweden', 'Colombia', 'Uruguay', 'Brazil', 'Sweden', 'Russia', 'France', 'Croatia', 'Belgium', 'France']


,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,...,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,winner_encoded,home_team_result
0,22,23,5,0,6,2,1,0,1,0,...,8,0,3,4,22,10,38,62,22,1.0
1,8,31,0,1,0,5,2,0,0,0,...,4,6,3,6,12,6,41,59,31,0.0
2,16,13,0,1,5,2,1,0,3,0,...,4,3,6,6,22,14,67,33,13,0.0
3,21,27,3,3,4,5,1,0,1,0,...,4,7,3,5,12,10,33,67,21,1.0
4,10,1,2,1,5,1,1,0,3,0,...,6,2,5,3,16,19,55,45,10,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,22,6,2,2,6,8,1,0,4,0,...,6,4,7,11,25,18,35,65,22,1.0
60,10,2,1,0,4,5,2,0,3,0,...,6,4,8,5,6,16,36,64,10,1.0
61,6,9,2,1,8,4,2,0,1,0,...,8,3,11,6,23,14,55,45,6,1.0
62,2,9,2,0,4,5,1,0,2,0,...,5,7,3,7,11,4,42,58,2,1.0


In [5]:
# Writeback to csv data file
dataFrame.to_csv('../Data/Dataset.csv',index=False)

In [6]:
dataFrame[0:48].to_csv('../Data/TrainingData.csv',index=False)